In [5]:
import pandas as pd

xls = pd.ExcelFile('../Lesson_3/isp_supply_chain.xlsx')

df_orders = xls.parse('Orders')
df_deliveries = xls.parse('Deliveries')
df_claims = xls.parse('Claims')

df_orders['PRICE'] = df_orders['PRICE'].apply(float)
df_orders.info()
df_orders.head()


<class 'pandas.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ORDER_ID    10 non-null     str           
 1   MSISDN      10 non-null     int64         
 2   PRODUCT     10 non-null     str           
 3   PRICE       10 non-null     float64       
 4   ORDER_DATE  10 non-null     datetime64[us]
dtypes: datetime64[us](1), float64(1), int64(1), str(2)
memory usage: 532.0 bytes


,ORDER_ID,MSISDN,PRODUCT,PRICE,ORDER_DATE
0,ORD00001,960000001,Router WiFi 6,2900.0,2026-01-01
1,ORD00002,960000002,Mesh Node,1500.0,2026-01-02
2,ORD00003,960000003,Fiber Modem,0.0,2026-01-03
3,ORD00004,960000004,CCTV Cloud,990.0,2026-01-04
4,ORD00005,960000005,Router WiFi 6,2900.0,2026-01-05


In [ ]:

print('ตรวจสอบค่าว่างในตารางการจัดส่ง')
df_deliveries.isna().sum()
df_deliveries['CARRIER'] = df_deliveries['CARRIER'].fillna('Unknown Customer')
df_deliveries['DELIVERY_DATE'] = df_deliveries['DELIVERY_DATE'].apply(pd.to_datetime)
df_deliveries.info()
df_deliveries

ตรวจสอบค่าว่างในตารางการจัดส่ง
<class 'pandas.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DELIVERY_ID    10 non-null     str           
 1   ORDER_ID       10 non-null     str           
 2   CARRIER        10 non-null     str           
 3   STATUS         10 non-null     str           
 4   DELIVERY_DATE  5 non-null      datetime64[us]
dtypes: datetime64[us](1), str(4)
memory usage: 532.0 bytes


,DELIVERY_ID,ORDER_ID,CARRIER,STATUS,DELIVERY_DATE
0,DLV00001,ORD00001,Kerry,Delivered,2026-01-05
1,DLV00002,ORD00002,Flash,Delivered,2026-01-06
2,DLV00003,ORD00003,J&T,Shipped,NaT
3,DLV00004,ORD00004,Kerry,Delivered,2026-01-08
4,DLV00005,ORD00005,Flash,Pending,NaT
5,DLV00006,ORD00006,Unknown Customer,Cancelled,NaT
6,DLV00007,ORD00007,J&T,Delivered,2026-01-10
7,DLV00008,ORD00008,Kerry,Shipped,NaT
8,DLV00009,ORD00009,Unknown Customer,Pending,NaT
9,DLV00010,ORD00010,Flash,Delivered,2026-01-12


In [19]:
print(df_claims.info())
df_claims


<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   CLAIM_ID      2 non-null      str  
 1   ORDER_ID      2 non-null      str  
 2   REASON        2 non-null      str  
 3   CLAIM_STATUS  2 non-null      str  
dtypes: str(4)
memory usage: 196.0 bytes
None


,CLAIM_ID,ORDER_ID,REASON,CLAIM_STATUS
0,CLM001,ORD00001,Power not on,Approved
1,CLM002,ORD00007,Broken Box,Pending


In [ ]:
df_merged = pd.merge(df_orders, df_deliveries, on='ORDER_ID', how='left')
df_merged


,ORDER_ID,MSISDN,PRODUCT,PRICE,ORDER_DATE,DELIVERY_ID,CARRIER,STATUS,DELIVERY_DATE
0,ORD00001,960000001,Router WiFi 6,2900.0,2026-01-01,DLV00001,Kerry,Delivered,2026-01-05
1,ORD00002,960000002,Mesh Node,1500.0,2026-01-02,DLV00002,Flash,Delivered,2026-01-06
2,ORD00003,960000003,Fiber Modem,0.0,2026-01-03,DLV00003,J&T,Shipped,NaT
3,ORD00004,960000004,CCTV Cloud,990.0,2026-01-04,DLV00004,Kerry,Delivered,2026-01-08
4,ORD00005,960000005,Router WiFi 6,2900.0,2026-01-05,DLV00005,Flash,Pending,NaT
5,ORD00006,960000006,Mesh Node,1500.0,2026-01-06,DLV00006,Unknown Customer,Cancelled,NaT
6,ORD00007,960000007,Router WiFi 6,2900.0,2026-01-07,DLV00007,J&T,Delivered,2026-01-10
7,ORD00008,960000008,Fiber Modem,0.0,2026-01-08,DLV00008,Kerry,Shipped,NaT
8,ORD00009,960000009,CCTV Cloud,990.0,2026-01-09,DLV00009,Unknown Customer,Pending,NaT
9,ORD00010,960000010,Mesh Node,1500.0,2026-01-10,DLV00010,Flash,Delivered,2026-01-12


In [25]:
df_final = pd.merge(df_merged, df_claims, on='ORDER_ID', how='left')
df_final['CLAIM_STATUS'] = df_final['CLAIM_STATUS'].fillna('No Claim')
df_final[['ORDER_ID','PRODUCT','PRICE','STATUS','CLAIM_STATUS']]

,ORDER_ID,PRODUCT,PRICE,STATUS,CLAIM_STATUS
0,ORD00001,Router WiFi 6,2900.0,Delivered,Approved
1,ORD00002,Mesh Node,1500.0,Delivered,No Claim
2,ORD00003,Fiber Modem,0.0,Shipped,No Claim
3,ORD00004,CCTV Cloud,990.0,Delivered,No Claim
4,ORD00005,Router WiFi 6,2900.0,Pending,No Claim
5,ORD00006,Mesh Node,1500.0,Cancelled,No Claim
6,ORD00007,Router WiFi 6,2900.0,Delivered,Pending
7,ORD00008,Fiber Modem,0.0,Shipped,No Claim
8,ORD00009,CCTV Cloud,990.0,Pending,No Claim
9,ORD00010,Mesh Node,1500.0,Delivered,No Claim


In [31]:
claims_only = df_final[df_final['CLAIM_STATUS'] != 'No Claim']
claims_only

claim_report = claims_only.groupby('PRODUCT')['PRICE'].agg(['count', 'sum'])
claim_report = claim_report.rename(columns={
    'count': 'จำนวนเคสที่เคลม',
    'sum': 'มูลค่าความเสียหายรวม'
})

claim_report = claim_report.style.format({
    'มูลค่าความเสียหายรวม': '{:,.2f} บาท' 
})

print('รายงานสรุปยอดเคลมแยกตามประเภทสินค้า:')
claim_report

รายงานสรุปยอดเคลมแยกตามประเภทสินค้า:


,จำนวนเคสที่เคลม,มูลค่าความเสียหายรวม
PRODUCT,,
Router WiFi 6,2,"5,800.00 บาท"
